In [10]:
import os
import json
import cv2
import numpy as np
import onnx
import onnxruntime
import onnx_graphsurgeon as gs

In [2]:
session = onnxruntime.InferenceSession("models/us_lprnet_baseline18_deployable.onnx")

In [13]:
print(session.get_inputs()[0].shape)
for output in session.get_outputs():
    print(output.name)
    print(output.type)
    print(output.shape)

['N', 3, 48, 96]
tf_op_layer_ArgMax
tensor(int32)
['N', 24]
tf_op_layer_Max
tensor(float)
['N', 24]


In [39]:
img = cv2.imread("images/liciense_plate.png", 1)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img[:img.shape[0] // 2]
img = cv2.resize(img, (96, 48))
cv2.imwrite("debug.jpg", img)
img = img.astype(np.float32)
img = img / 255
img = img.transpose(2, 0, 1)
img = np.expand_dims(img, 0)

In [40]:
outputs = session.run(None, {session.get_inputs()[0].name: img})

In [41]:
outputs

[array([[35, 35, 35, 35, 35, 35, 35, 35, 35, 35,  7, 35, 27,  9, 35, 35,
         35, 35, 35, 35, 35, 35, 35, 23]], dtype=int32),
 array([[0.99999905, 1.        , 1.        , 1.        , 1.        ,
         1.        , 0.99999964, 0.9999994 , 0.99999917, 0.97942334,
         0.48410153, 0.99961555, 0.43506563, 0.5636416 , 0.9772284 ,
         0.9999893 , 0.9999994 , 0.9999994 , 0.99999845, 0.99999714,
         0.99999774, 0.9999976 , 0.99960524, 0.32400212]], dtype=float32)]

In [42]:
def parse_custom_nvplate(output_layers_info, network_info):
    """Parse the NVPlate output to extract attributes and string."""
    with open("dict.txt", "r", encoding="utf-8") as f:
        dict_table = [line.strip() for line in f if line.strip()]

    output_str_buffer = None
    output_conf_buffer = None
    attr_string = ""

    seq_len = network_info["width"] // 4

    for layer_info in output_layers_info:
        if layer_info["dataType"] == "float32":
            output_conf_buffer = np.array(layer_info["buffer"], dtype=np.float32)
        elif layer_info["dataType"] == "int32":
            output_str_buffer = np.array(layer_info["buffer"], dtype=np.int32)

    str_idxes = []
    prev = -1
    bank_softmax_max = []

    for seq_id in range(seq_len):
        curr_data = output_str_buffer[seq_id]

        if curr_data < 0 or curr_data >= len(dict_table):
            continue

        if seq_id == 0 or curr_data != prev:
            str_idxes.append(curr_data)
            if curr_data != len(dict_table):  # Not padding
                bank_softmax_max.append(output_conf_buffer[seq_id])

        prev = curr_data

    for idx in str_idxes:
        if idx != len(dict_table):
            attr_string += dict_table[idx]

    attr_confidence = np.prod(bank_softmax_max)

    return True, attr_confidence, attr_string


output_layers_info = [
    {
        "dataType": "int32",
        "buffer": outputs[0][0],
    },
    {
        "dataType": "float32",
        "buffer": outputs[1][0],
    },
]
network_info = {"width": 24 * 4}

success, attributes, plate_string = parse_custom_nvplate(
    output_layers_info, network_info
)

if success:
    print("Attributes:", attributes)
    print("Plate String:", plate_string)
else:
    print("Parsing failed.")

Attributes: 0.038462907
Plate String: 7S9N


In [1]:
import cv2
import numpy as np
from scrfd import SCRFD

In [2]:
img_path = "images/face_img.jpg"
face_detector = SCRFD("models/scrfd.onnx")
boxes, kpss, scores = face_detector.detect(img_path)
boxes = boxes[0].astype(int)
kpss = kpss[0].astype(int)

Warming up ...
Forwarding time: 0.02313709259033203
Forwarding time: 0.016562461853027344


In [4]:
img = cv2.imread(img_path, 1)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.circle(img, kpss[0].tolist(), radius=10, color=(0, 0, 255), thickness=-1)
img = cv2.circle(img, kpss[1].tolist(), radius=10, color=(0, 0, 255), thickness=-1)
cv2.imwrite("debug.jpg", img)

True

In [ ]:
left_eye, right_eye = kpss[:2]
if left_eye[1] > right_eye[1]:  # mắt trái thấp hơn mắt phải
    point_A = (x_right_eye, y_left_eye)
    # Ảnh phải xoay theo chiều kim đồng hộ clockwise direction
    direction = -1
else:
    point_A = (x_left_eye, y_right_eye)
    # Ảnh cần xoay ngược chiều kim đồng hồ counter clockwise direction
    direction = 1